<a href="https://colab.research.google.com/github/Anjali-042/AI-LAB/blob/main/resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# A function to perform the resolution operation between two clauses
def resolve(clause1, clause2):
    """
    Resolves two clauses and returns the new clause formed by resolving them.
    If no resolution is possible, returns None.
    """
    resolved = set()
    for literal in clause1:
        if -literal in clause2:
            # If complementary literals are found (e.g., 'p' and '-p'), remove them and merge the rest
            new_clause = (clause1 - {literal}) | (clause2 - {-literal})
            resolved.update(new_clause)
            break  # Once we find the resolvent, no need to check further literals
    else:
        return None  # No resolvable pair found
    return resolved

# Function to build a resolution tree
def build_resolution_tree(clauses):
    """
    Builds a resolution tree from a list of clauses.
    It returns a resolution tree as a dictionary where keys are the resolvent clauses and
    values are the list of parent clauses that generated it.
    """
    tree = {}
    queue = list(clauses)

    while queue:
        clause1 = queue.pop(0)  # Get the first clause

        for clause2 in queue:  # Compare it with the rest
            resolvent = resolve(clause1, clause2)

            if resolvent:
                if frozenset(resolvent) not in tree:
                    tree[frozenset(resolvent)] = [(frozenset(clause1), frozenset(clause2))]
                    queue.append(resolvent)  # Add the new resolvent to the queue
                else:
                    tree[frozenset(resolvent)].append((frozenset(clause1), frozenset(clause2)))

    return tree

# A helper function to visualize the resolution tree
def print_resolution_tree(tree):
    """
    Prints the resolution tree in a readable format.
    """
    for resolvent, parents in tree.items():
        print(f"Resolvent: {resolvent}")
        for parent1, parent2 in parents:
            print(f"  Parent clauses: {parent1} and {parent2}")
        print()

# Example clauses in Conjunctive Normal Form (CNF)
clauses = [
    {mother(leela,Oshin)},   # (p OR q)
    {-mother(x,y) OR parent(x,y)},  # (¬p OR r)
    {-parent(x,y) OR alive(x) OR older(x,y)},
    {alive(leela)},
    {-older(leela,Oshin)}# (¬q OR ¬r)
]

# Build the resolution tree
tree = build_resolution_tree(clauses)

# Print the resolution tree
print_resolution_tree(tree)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-3-8e5b8ec6bd21>, line 57)

In [8]:
from typing import List, Tuple

def is_complementary(clause1: set, clause2: set) -> Tuple[bool, set]:
    """
    Check if two clauses are complementary and return the resolved clause if they are.
    """
    for literal in clause1:
        if f"~{literal}" in clause2:
            resolved = clause1.union(clause2)
            resolved.discard(literal)
            resolved.discard(f"~{literal}")
            return True, resolved
        elif literal.startswith("~") and literal[1:] in clause2:
            resolved = clause1.union(clause2)
            resolved.discard(literal)
            resolved.discard(literal[1:])
            return True, resolved
    return False, set()

def resolution(kb: List[set], goal: set) -> Tuple[bool, List[Tuple[set, List[int]]]]:
    """
    Perform resolution and build the proof tree.
    """
    proof_tree = []
    clauses = kb + [goal]  # Add negated goal to the KB
    new_clauses = []

    while True:
        # Generate new pairs of clauses
        pairs = [(clauses[i], clauses[j], (i, j)) for i in range(len(clauses)) for j in range(i + 1, len(clauses))]

        for clause1, clause2, (i, j) in pairs:
            complementary, resolved_clause = is_complementary(clause1, clause2)
            if complementary:
                if not resolved_clause:  # Empty clause found
                    proof_tree.append((resolved_clause, [i, j]))
                    return True, proof_tree
                if resolved_clause not in clauses and resolved_clause not in new_clauses:
                    new_clauses.append(resolved_clause)
                    proof_tree.append((resolved_clause, [i, j]))

        if not new_clauses:  # No progress, resolution failed
            return False, proof_tree

        clauses.extend(new_clauses)
        new_clauses = []

# Example case study
if __name__ == "__main__":
    # Knowledge Base (KB) in CNF format
    kb = [
        {"P", "Q"},       # Clause 1: P ∨ Q
        {"~P", "R"},      # Clause 2: ¬P ∨ R
        {"~R"},           # Clause 3: ¬R
    ]
    goal = {"~Q"}  # Negated goal: ¬Q

    success, proof_tree = resolution(kb, goal)
    if success:
        print("PROVED")
    else:
        print("CAN'T BE PROVED")

    print("\nProof Tree:")
    for i, (clause, parents) in enumerate(proof_tree):
        print(f"Step {i + 1}: {clause} from {parents}")


PROVED

Proof Tree:
Step 1: {'R', 'Q'} from [0, 1]
Step 2: {'P'} from [0, 3]
Step 3: {'~P'} from [1, 2]
Step 4: {'Q'} from [0, 6]
Step 5: {'R'} from [1, 5]
Step 6: set() from [5, 6]
